In [10]:
cols = ['Loan Type:',
'Loan Award ID:',
'Loan Attending School Name:',
'Loan Date:',
'Loan Amount:',
'Loan Disbursed Amount:',
'Loan Outstanding Principal Balance:',
'Loan Outstanding Interest Balance:',
'Loan Interest Rate Type:',
'Loan Interest Rate:',
'Loan Contact Name:']

In [11]:
delimiter = 'Loan Type:'
loans = open('MyStudentData.txt', 'r').read().split(delimiter)

In [12]:
loans.pop(0) # remove header info

'File Source:U.S. DEPARTMENT OF EDUCATION, NATIONAL STUDENT LOAN DATA SYSTEM (NSLDS)\nFile Request Date:2020-09-28-05.53.32.292\nStudent First Name:MARIA\nStudent Middle Initial:\nStudent Last Name:BABAKHANYAN\nStudent Street Address 1:123 EAST SAN CARLOS STREET #266\nStudent Street Address 2:\nStudent City:SAN JOSE\nStudent State Code:CA\nStudent Country Code:US\nStudent Zip Code:95112\nStudent Email Address:sndik16@GMAIL.COM\nStudent Home Phone Country Code:1\nStudent Home Phone Number:4159176460\nStudent Home Phone Preferred:No\nStudent Cell Phone Country Code:1\nStudent Cell Phone Number:8145315055\nStudent Cell Phone Preferred:Yes\nStudent Work Phone Country Code:\nStudent Work Phone Number:\nStudent Work Phone Preferred:\nStudent SULA Maximum Eligibility Period:0.0\nStudent SULA Subsidized Usage Period:0.0\nStudent SULA Remaining Eligibility Period:0.0\nStudent Enrollment Status:GRADUATED\nStudent Enrollment Status Effective Date:05/16/2017\nStudent Total All Loans Outstanding Pr

In [13]:
# devide by each loan
proc_loans = []
for l in loans:
    proc_loans.append((delimiter + l).split('\n'))

In [14]:
# rangle data into consistent format for pandas
data = []
for p in proc_loans:
    temp_loan = []
    for c in cols:
        for line in p:
            if line.startswith(c):
                # strip dollar signs and commas and % so they end up as floats
                temp_loan.append(line.split(':')[1].replace('$', '').replace(',', '').replace('%', ''))
    # HACK: pull off extra loan contact names and only retain the first one
    if len(temp_loan) > 11:
        for n in range(len(temp_loan) - 11):
            temp_loan.pop()
    data.append(temp_loan)
# data

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame.from_records(data, columns=list(map(lambda c: c.replace(':', ''), cols)), coerce_float=True)

In [17]:
# convert date
df['Loan Date'] = df['Loan Date'].astype('datetime64')
# convert floats
cols_to_floats = [
    'Loan Amount',
    'Loan Disbursed Amount',
    'Loan Outstanding Principal Balance',
    'Loan Outstanding Interest Balance',
#     'Loan Interest Rate:' # NOTE: there are blanks in my data, so would need to account for this to convert to a float
]
for c in cols_to_floats:
    df[c] = df[c].astype('float')

In [18]:
df.describe()

,Loan Amount,Loan Disbursed Amount,Loan Outstanding Principal Balance,Loan Outstanding Interest Balance
count,25.000000,25.000000,25.000000,25.000000
mean,6908.360000,6908.360000,7076.120000,2214.560000
std,6064.693314,6064.693314,6066.422864,2607.676298
min,265.000000,265.000000,265.000000,0.000000
25%,2791.000000,2791.000000,2791.000000,159.000000
50%,5500.000000,5500.000000,5560.000000,1185.000000
75%,7584.000000,7584.000000,8349.000000,3810.000000
max,20500.000000,20500.000000,20500.000000,10012.000000


In [19]:
df.to_csv('output.csv', index = True)